# 2x2 Rubik Cube Solver

# 1. Setting Up

## 1.1. Libraries

In [25]:
# data manipulation
import numpy as np

# utilities
import random
import math
from queue import PriorityQueue

# images
from PIL import Image, ImageChops

## 1.2. Constants

In [26]:
# U: Up
# F: Front
# R: Right
# L: Left
# B: Back
# D: Down
# Single quote (') means counterclockwise rotation (e.g. U' is up counterclockwise)
MOVES = ("U", "D", "R", "L", "F", "B", "U'", "D'", "R'", "L'", "F'", "B'")
MOVES_COLORS = {
    "U": 'yellow-right',
    "D": 'white-right',
    "R": 'blue-right',
    "L": 'green-right',
    "F": 'orange-right',
    "B": 'red-right',
    "U'": 'yellow-left',
    "D'": 'white-left',
    "R'": 'blue-left',
    "L'": 'green-left',
    "F'": 'orange-left',
    "B'": 'red-left',
}

# UUUU.DDDD.RRRR.LLLL.FFFF.BBBB (up.down.right.left.front.back)
# YYYY.WWWW.BBBB.GGGG.OOOO.RRRR (yellow.white.blue.green.orange.red)
SOLVED_CUBE = "YYYYWWWWBBBBGGGGOOOORRRR"

# Hexadecimal base colors
COLORS = {
    "Y": "#f6fe11",
    "G": "#08800c",
    "O": "#faaf0d",
    "B": "#0049e6",
    "R": "#f61109",
    "W": "#f0f0f0"
};

## 1.3. Rotations

In [27]:
def rotate_cube(cube, move):
	if move not in MOVES:
		print(move)
		raise Exception('Mal movimiento de entrada')

	new_cube = list(cube)

	# UUUU.DDDD.RRRR.LLLL.FFFF.BBBB (up.down.right.left.front.back)
  # YYYY.WWWW.BBBB.GGGG.OOOO.RRRR (yellow.white.blue.green.orange.red)

	if move == "U":
		# sides
		# back -> right
		new_cube[8], new_cube[9] = cube[23], cube[22]
		# left -> back
		new_cube[23], new_cube[22] = cube[12], cube[13]
		# front -> left
		new_cube[12], new_cube[13] = cube[16], cube[17]
		# right -> front
		new_cube[16], new_cube[17] = cube[8], cube[9]
		# turn the top face
		new_cube[0], new_cube[1], new_cube[2], new_cube[3] = cube[2], cube[0], cube[3], cube[1]
	elif move == "D":
		# sides
		# back -> left
		new_cube[14], new_cube[15] = cube[20], cube[21]
		# left -> front
		new_cube[18], new_cube[19] = cube[14], cube[15]
		# front -> right
		new_cube[10], new_cube[11] = cube[18], cube[19]
		# right -> back
		new_cube[20], new_cube[21] = cube[10], cube[11]
		# turn the down face
		new_cube[4], new_cube[5], new_cube[6], new_cube[7] = cube[6], cube[4], cube[7], cube[5]
	elif move == "R":
		# sides
		# front -> top
		new_cube[3], new_cube[1] = cube[17], cube[19]
		# top -> back
		new_cube[23], new_cube[21] = cube[3], cube[1]
		# back -> bottom
		new_cube[7], new_cube[5] = cube[23], cube[21]
		# bottom -> front
		new_cube[17], new_cube[19] = cube[7], cube[5]
		# turn the right face
		new_cube[8], new_cube[9], new_cube[10], new_cube[11] = cube[10], cube[8], cube[11], cube[9]
	elif move == "L":
		# sides
		# front -> down
		new_cube[6], new_cube[4] = cube[18], cube[16]
		# top -> front
		new_cube[18], new_cube[16] = cube[2], cube[0]
		# back -> top
		new_cube[2], new_cube[0] = cube[22], cube[20]
		# down -> back
		new_cube[22], new_cube[20] = cube[6], cube[4]
		# turn the left face
		new_cube[12], new_cube[13], new_cube[14], new_cube[15] = cube[14], cube[12], cube[15], cube[13]
	elif move == "F":
		# sides
		# top -> right
		new_cube[8], new_cube[10] = cube[2], cube[3]
		# left -> up
		new_cube[2], new_cube[3] = cube[15], cube[13]
		# bottom -> left
		new_cube[15], new_cube[13] = cube[5], cube[4]
		# right -> down
		new_cube[5], new_cube[4] = cube[8], cube[10]
		# turn the front face
		new_cube[16], new_cube[17], new_cube[18], new_cube[19] = cube[18], cube[16], cube[19], cube[17]
	elif move == "B":
		# sides
		# top -> left
		new_cube[14], new_cube[12] = cube[0], cube[1]
		# left -> down
		new_cube[7], new_cube[6] = cube[14], cube[12]
		# down -> right
		new_cube[9], new_cube[11] = cube[7], cube[6]
		# right -> up
		new_cube[0], new_cube[1] = cube[9], cube[11]
		# turn the back face
		new_cube[20], new_cube[21], new_cube[22], new_cube[23] = cube[22], cube[20], cube[23], cube[21]
	elif move == "U'":
		# sides
		# right -> back
		new_cube[23], new_cube[22] = cube[8], cube[9]
		# back -> left
		new_cube[12], new_cube[13] = cube[23], cube[22]
		# left -> front
		new_cube[16], new_cube[17] = cube[12], cube[13]
		# front -> right
		new_cube[8], new_cube[9] = cube[16], cube[17]
		# turn the top face
		new_cube[2], new_cube[0], new_cube[3], new_cube[1] = cube[0], cube[1], cube[2], cube[3]
	elif move == "D'":
		# sides
		# left -> back
		new_cube[20], new_cube[21] = cube[14], cube[15]
		# front -> left
		new_cube[14], new_cube[15] = cube[18], cube[19]
		# right -> front
		new_cube[18], new_cube[19] = cube[10], cube[11]
		# back -> right
		new_cube[10], new_cube[11] = cube[20], cube[21]
		# turn the down face
		new_cube[6], new_cube[4], new_cube[7], new_cube[5] = cube[4], cube[5], cube[6], cube[7]
	elif move == "R'":
		# sides
		# top -> front
		new_cube[17], new_cube[19] = cube[3], cube[1]
		# back -> top
		new_cube[3], new_cube[1] = cube[23], cube[21]
		# down -> back
		new_cube[23], new_cube[21] = cube[7], cube[5]
		# front -> bottom
		new_cube[7], new_cube[5] = cube[17], cube[19]
		# turn the right face
		new_cube[10], new_cube[8], new_cube[11], new_cube[9] = cube[8], cube[9], cube[10], cube[11]
	elif move == "L'":
		# sides
		# bottom -> front
		new_cube[18], new_cube[16] = cube[6], cube[4]
		# front -> top
		new_cube[2], new_cube[0] = cube[18], cube[16]
		# top -> back
		new_cube[22], new_cube[20] = cube[2], cube[0]
		# back -> down
		new_cube[6], new_cube[4] = cube[22], cube[20]
		# turn the left face
		new_cube[14], new_cube[12], new_cube[15], new_cube[13] = cube[12], cube[13], cube[14], cube[15]
	elif move == "F'":
		# sides
		# right -> up
		new_cube[2], new_cube[3] = cube[8], cube[10]
		# top -> left
		new_cube[15], new_cube[13] = cube[2], cube[3]
		# left -> down
		new_cube[5], new_cube[4] = cube[15], cube[13]
		# down -> right
		new_cube[8], new_cube[10] = cube[5], cube[4]
		# turn the front face
		new_cube[18], new_cube[16], new_cube[19], new_cube[17] = cube[16], cube[17], cube[18], cube[19]
	elif move == "B'":
		# sides
		# left -> up
		new_cube[0], new_cube[1] = cube[14], cube[12]
		# down -> left
		new_cube[14], new_cube[12] = cube[7], cube[6]
		# right -> down
		new_cube[7], new_cube[6] = cube[9], cube[11]
		# top -> right
		new_cube[9], new_cube[11] = cube[0], cube[1]
		# turn the back face
		new_cube[22], new_cube[20], new_cube[23], new_cube[21] = cube[20], cube[21], cube[22], cube[23]

	return ''.join(new_cube)

# generate random cube
def random_cube(num_mov=10):
  cube = SOLVED_CUBE
  moves = []
  for _ in range(num_mov):
    # random.seed(0)
    move = MOVES[random.randint(0, len(MOVES) - 1)]
    moves.append(move)
    cube = rotate_cube(cube, move)
  return cube, moves

# For function cubeStr2Matrix
section_to_pos = {
    0: (2, 0),
    1: (2, 4),
    2: (4, 2),
    3: (0, 2),
    4: (2, 2),
    5: (6, 2),
}

# convert cube from string to matrix
def cubeStr2Matrix(cube):
  m = np.empty((6, 8), dtype='U')
  m[:] = ' '
  for i in range(len(cube)):
    sec = i // 4
    mod = i % 4
    x, y = section_to_pos[sec]
    if sec != 5:
      m[y + (mod >= 2)*1, x + (mod % 2)] = cube[i]
    else:
      m[(y + 1) - (mod >= 2)*1, (x + 1) - (mod % 2)] = cube[i]
  return m

## 1.4. Image Reading

In [28]:
import math # import math module for mathematical operations

# hexadecimal to RGB conversion function
def hex_to_rgb(hex):
  if hex[0] == '#':
    hex = hex[1:]
  # return the resulting 3 integers representing RGB values
  return tuple(int(hex[i:i + 2], 16) for i in (0, 2, 4));

# function to calculate the distance between 2 colors in RGB space
def colors_dist(color1, color2):
  r1, g1, b1 = color1
  r2, g2, b2 = color2
  # calculate the Euclidean distance between the 2 colors in RGB space
  return math.sqrt((r1 - r2) ** 2 + (g1 - g2) ** 2 + (b1 - b2) ** 2)

# function to identify the most similar color from a list of predefined colors
def most_similar_color(color, margin):
  max_dist = math.sqrt(255 ** 2 + 255 ** 2 + 255 ** 2) # maximum possible distance between any 2 colors in RGB space
  min_dist = math.inf # set the minimum distance initially to infinity
  color_name = None # set the color name initially to None
  color1 = hex_to_rgb(color)
  # iterate through the predefined colors to find the most similar color
  for c_name in COLORS:
    color2 = hex_to_rgb(COLORS[c_name])
    dist = colors_dist(color1, color2)
    # compare the distance with the specified margin and the previous minimum distance
    if dist / max_dist < margin and dist < min_dist:
      # if the distance is less than the margin and less than the previous minimum distance, update the minimum distance and the name of the most similar color found so far
      min_dist = dist
      color_name = c_name
  return color_name # return the name of the most similar color found

In [29]:
# Eliminate image padding
# taken from https://stackoverflow.com/questions/10615901/trim-whitespace-using-pil/10616717#10616717
def trim(im):
  bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
  diff = ImageChops.difference(im, bg)
  diff = ImageChops.add(diff, diff, 2.0, -100)
  bbox = diff.getbbox()
  return im.crop(bbox) if bbox else im

def read_image(file_path):
  image = Image.open(file_path)
  image = trim(image)
  image.show()

  # Get image size
  width, height = image.size

  # Get pixel matrix
  pixeles = image.load()

  # Dictionary of top-left corner positions for each face of the Rubik's cube
  # UUUU.DDDD.RRRR.LLLL.FFFF.BBBB (up.down.right.left.front.back)
  top_left_corners = {
      'U': (2, 0), # position in image matrix
      'D': (2, 4),
      'R': (4, 2),
      'L': (0, 2),
      'F': (2, 2),
      'B': (6, 2)
  }

  cube = ''
  # Iterate through each face of the cube
  for face_name in top_left_corners:
    # Get top-left position of the current face
    x, y = top_left_corners[face_name]
    # Reverse order of colors for back face
    rev = face_name == 'B'
    face = ''
    # Iterate through each pixel of the current face
    for i in range(2):
      for j in range(2):
        # Get color of current pixel and find the most similar color in our predefined list of colors
        color_pixel = pixeles[(x+j)*width/8+width/16, (y+i)*height/6+height/12]
        color = '#%02x%02x%02x' % (color_pixel[0], color_pixel[1], color_pixel[2])
        if not rev: # normal case
          face += most_similar_color(color, 10)
        else: # back face
          face = most_similar_color(color, 10) + face
    # Append the colors of the current face to the cube string
    cube += face
  return cube

# 2. Heuristic

## 2.1. Heuristic Function

In [30]:
def heuristic1(cube):
    # Count the number of out-of-place tiles
  out_of_place = 0
  for i in range(len(cube)):
    if cube[i] != SOLVED_CUBE[i]:
      out_of_place += 1

  return out_of_place

def heuristic2(cube):
  # Calculate the Manhattan distance for each tile
  distance = 0
  for i in range(len(cube)):
    if cube[i] != SOLVED_CUBE[i]:
      distance += abs(i // 4 - SOLVED_CUBE.index(cube[i]) // 4) + abs(i % 4 - SOLVED_CUBE.index(cube[i]) % 4)
  return distance

def heuristic4(cube):
  # Define the solved corners permutation
  solved_corners = [(0, 2, 5), (0, 4, 7), (1, 3, 6), (1, 4, 6)]

  # Count the number of corners that are in the correct position relative to each other
  correct_corners = 0
  for corner in solved_corners:
    if all(cube[i] == SOLVED_CUBE[j] for i, j in zip(corner, corner)):
      correct_corners += 1

  # Returns the number of corners that are not in the correct position
  return 4 - correct_corners

def heuristic5(cube):
  # Count the number of edges that are not oriented correctly
  incorrect_edges = 0
  for i, j in [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]:
    if cube[i] != SOLVED_CUBE[i] and cube[j] != SOLVED_CUBE[j]:
      incorrect_edges += 1

  # Returns the number of edges that are not oriented correctly
  return incorrect_edges

## 2.2. Solver Algorithm

In [31]:
# tree
class Node:
  def __init__(self, state, step, heur, dist, parent):
    self.state = state # estado del cubo
    self.step = step # movimiento que llevó a este nodo (None para la raíz)
    self.heur = heur # valor heurístico del nodo
    self.dist = dist # distancia o costo para llegar a este nodo desde la raíz
    self.children = [] # lista de hijos del nodo
    self.parent = parent # padre del nodo
    self.cost_func = heur + dist # función de costo

  def __str__(self):
    return str(self.step) + ': ' + self.state + ' (h=' + str(self.heur) + ', c=' + str(self.dist) + ')'

  # cadena de retorno para la vista de árbol
  def tree(self, level=0):
    ret = "\t" * level + str(self) + "\n"
    for child in self.children:
      ret += child.tree(level + 1)
    return ret

  # para comparar en la PriorityQueue
  def __lt__(self, other):
    return self.cost_func < other.cost_func

# búsqueda A*
def solve_cube(cube, max_it=10000, keep_searching=False):
  # Para salida en .txt
  # report = f'ESTADO INICIAL (h={})'
  report = 'RESULTS\n\n1. Algorithm Traversal\n(Note: s=current state, h=heuristic, d=distance)\n'

  # Raíz del árbol
  root = Node(cube, None, heuristic1(cube), 0, None)
  best_node = root
  report += f'INITIAL STATE:\ns = {root.state}\n'

  # Inicializar la cola de prioridad para la búsqueda (menor puntuación f -> mayor prioridad)
  queue = PriorityQueue()
  queue.put(root)

  # Para guardar estados visitados y no volver a ellos
  visited = set()

  # Contador de iteraciones
  num_it = 0

  # bucle principal de la búsqueda
  while not queue.empty() and num_it < max_it:
    num_it += 1;

    # Obtener el siguiente nodo de la cola de prioridad
    curr_node = queue.get()
    report += f'ITERATION {num_it}:\ns = {curr_node.state}, h = {curr_node.heur}, d = {curr_node.dist}\n'

    # Comprobar si el cubo está resuelto
    if curr_node.heur == 0:
      # report += 'SOLUTION FOUND\n'
      best_node = curr_node
      if not keep_searching: break

    # Genera todos los posibles nodos hijos
    report += 'Generating children...\n'
    for move in MOVES:
      new_cube = rotate_cube(curr_node.state, move)
      if new_cube not in visited:
        visited.add(new_cube)
        new_node = Node(new_cube, move, heuristic1(new_cube), curr_node.dist + 1, curr_node);
        report += f'Child: s = {new_cube}, cost_function = {new_node.cost_func}\n'
        curr_node.children.append(new_node)

        # Añade el nodo hijo a la cola de prioridad
        queue.put(new_node)

  return num_it, best_node, root, report

# 3. Input

In [32]:
# OPTION 1: FROM RANDOMLY GENERATED CUBE
# num_mov = 5 # number of random initial movements
# cube, random_moves = random_cube(num_mov)
# print('Randomly generated moves', random_moves)

# OPTION 2: FROM A PNG IMAGE
# Use site https://rubiks-cube-solver.com/2x2/ to take a screenshot of the cube. Save the image in the workspace.
img_path = 'cube.png' # image path
cube = read_image(img_path)

# 4. Output

In [33]:
# Input cube representation
cube

'RBYOOROGBRBWYOGGBWYWWRGY'

In [34]:
max_it = 100000000; # Maximum number of iterations

# If keep_searching=True, then it will keep searching even if it has found a solution, in order to find a better one.
num_it, best_node, root, report = solve_cube(cube, max_it, keep_searching=False);

In [35]:
# Build the sequence of moves that solved the cube (from the end to the beginning; and then the other way around)
moves = []
states = []
if best_node.heur == 0:
  node = best_node
  while node.step is not None: # While the node is the root
    moves.append(node.step)
    states.append(node.state)
    node = node.parent
  moves.reverse()
  states.reverse()

  report += '\n2. Steps to Solve the Cube:\n'
  report += f'Initial state:\n{cubeStr2Matrix(root.state)}\n'
  for i in range(len(states)):
    report += f'{MOVES_COLORS[moves[i]]}:\n{cubeStr2Matrix(states[i])}\n'
else:
  report += '\n2. No Solution Found\n'

In [36]:
print('RESULTS:\n')

if best_node.heur == 0:
  print('Solution found')
  print(moves)
else:
  print('Solution not found:')

print('Heuristics taken:', num_it)
print('Lowest heuristic achieved:', best_node.heur)
print('Cost of the best node:', best_node.dist)
print('Nearest cube representation:', best_node.state)

RESULTS:

Solution found
["L'", "B'", "R'"]
Heuristics taken: 31
Lowest heuristic achieved: 0
Cost of the best node: 3
Nearest cube representation: YYYYWWWWBBBBGGGGOOOORRRR


In [37]:
# REPORT

# Report will be saved on report.txt in the workspace
f = open('report.txt', 'w')
f.write(report)
f.close()

In [38]:
print(report)

RESULTS

1. Algorithm Traversal
(Note: s=current state, h=heuristic, d=distance)
INITIAL STATE:
s = RBYOOROGBRBWYOGGBWYWWRGY
ITERATION 1:
s = RBYOOROGBRBWYOGGBWYWWRGY, h = 18, d = 0
Generating children...
Child: s = YROBOROGYGBWBWGGBRYWWROY, cost_function = 20
Child: s = RBYOOOGRBRYWYOWRBWGGBWGY, cost_function = 23
Child: s = RWYWOROYBBWRYOGGBGYRWBGO, cost_function = 20
Child: s = WBGOBRYGBRBWGYGORWYWOROY, cost_function = 20
Child: s = RBGOBBOGYROWYOGRYBWWWRGY, cost_function = 23
Child: s = RWYOORYGBGBOBORGBWYWGWYR, cost_function = 20
Child: s = BORYOROGBWBWYGGGYOYWWRRB, cost_function = 16
Child: s = RBYORGOOBRWRYOYWBWBWGGGY, cost_function = 23
Child: s = RRYYOWOWRWBBYOGGBOYBWRGG, cost_function = 15
Child: s = BBYOWRGGBRBWOGYGOWOWRRYY, cost_function = 15
Child: s = RBBBOGOGRROWYOGYWWBYWRGY, cost_function = 23
Child: s = GYYOORWRBRBBOOGGBWYWRYWG, cost_function = 16
ITERATION 2:
s = RRYYOWOWRWBBYOGGBOYBWRGG, h = 14, d = 1
Generating children...
Child: s = YRYROWOWGGBBBOGGRWYBWROY, cost_f